In [0]:
from os import path
from pyspark.sql.functions import *

# PZ

In [0]:
pz_path = "/mnt/prelandingzone/Brazil/DirectSellout/RedeMateus"

dbutils.fs.ls(pz_path)

In [0]:
rede_mateus_2021_pz_path = path.join(pz_path, "2021**********")
rede_mateus_2022_pz_path = path.join(pz_path, "2022**********")
rede_mateus_2023_pz_path = path.join(pz_path, "2023**********")
rede_mateus_2024_pz_path = path.join(pz_path, "2024**********")

In [0]:
df_pz_2021 = spark.read.format("avro").option("mergeSchema", "true").load(rede_mateus_2021_pz_path)
df_pz_2022 = spark.read.format("avro").option("mergeSchema", "true").load(rede_mateus_2022_pz_path)
df_pz_2023 = spark.read.format("avro").option("mergeSchema", "true").load(rede_mateus_2023_pz_path)
df_pz_2024 = spark.read.format("avro").option("mergeSchema", "true").load(rede_mateus_2024_pz_path).where(year(col("DATA")) == 2023)

In [0]:
df_rede_mateus = df_pz_2021.union(df_pz_2022).union(df_pz_2023).union(df_pz_2024)

In [0]:
df_rede_mateus = df_rede_mateus.drop(col("filename"))

In [0]:
df_rede_mateus = df_rede_mateus.dropDuplicates()

In [0]:
df_rede_mateus = ( 
  df_rede_mateus
    .withColumn("year", year(col("DATA")))
    .withColumn("month", month(col("DATA")))
    .withColumn("day", when(dayofmonth(col("DATA")) < 16, lit(1)).otherwise(lit(16)))
)

In [0]:
df_rede_mateus.printSchema()

In [0]:
# path = "abfss://bronze@brewdatsazbrzp.dfs.core.windows.net/data/saz/br/sales/rede_mateus/sellout"

# (
#   df_rede_mateus
#     .write.format("delta")
#     .mode("overwrite")
#     .save("/mnt/delta/rede_mateus")
#     .partitionBy("year", "month", "day")
#     .option("partitionOverwriteMode", "dynamic")

# )

# HZ

In [0]:
historyzone_path = '/mnt/historyzone/Brazil/DirectSellout/RedeMateus'

dbutils.fs.ls(historyzone_path)

In [0]:
df_hz_rede_mateus = spark.read.format("parquet").load(historyzone_path).where(col("updatedDate") == '')

In [0]:
df_hz_rede_mateus = df_hz_rede_mateus.where(year(col("DATA")) <= 2023)

In [0]:
df_hz_rede_mateus = df_hz_rede_mateus.drop('filename', 'createdDate', 'updatedDate', 'lastReceived')

In [0]:
df_hz_rede_mateus = ( 
  df_hz_rede_mateus
    .withColumn("year", year(col("DATA")))
    .withColumn("month", month(col("DATA")))
    .withColumn("day", when(dayofmonth(col("DATA")) < 16, lit(1)).otherwise(lit(16)))
)

In [0]:
(
  df_hz_rede_mateus
    .groupBy("year", "month", "day")
    .agg(count("*").alias("count_rows"))
    .sort("year", "month", "day").display()
)

In [0]:
# silver_path = "abfss://silver@brewdatsazslvp.dfs.core.windows.net/data/saz/br/sales/rede_mateus/sellout"

# (
#   df_hz_rede_mateus
#     .write.mode("overwrite")
#     .partitionBy("year, month, day")
#     .option("partitionOverwriteMode", "dynamic")
#     .format("delta").save(silver_path)
# )
